## 대입식을 사용해 반복을 피하라

대입식은 영어로 assignment expression 이며 왈러스 연산자(walrus operator) 라고도 부른다.
- 파이썬 언어에서 고질적인 코드 중복 문제를 해결하고자 3.8에서 도입된 구문
- 일반 대입문은 `a=b` 라고 쓰며 `a equal b` 라고 읽지만...
- 왈러스 연산자는 `a:=b` 라고 쓰며, `a 왈러스 b` 라고 읽는다.

대입식은 대입문이 쓰일 수 없는 위치에서 변수에 값을 대입할 수 있으므로 유용하다.
- 예를 들어 if 문의 조건식 안에서 대입식을 쓸 수 있다. 대입식의 값은 왈러스 연산자 왼쪽에 있는 식별자에 대입된 값으로 평가된다.

In [1]:
fresh_fruit = {
    '사과': 10,
    '바나나': 8,
    '레몬': 5
}


# 고객이 레몬네이드를 주문했다면, 과즙을 낼 레몬이 과일 바구니에 최소 하나는 있어야 한다.
# 다음은 레몬의 개수를 읽어와서 그 값이 0이 아닌지 검사하는 코드

def make_lemonade(count):
    ...
    
def out_of_stock():
    ...
    

count = fresh_fruit.get('레몬', 0)
if count:
    make_lemonade(count)
else:
    out_of_stock()

필요 이상으로 잡음이 많은 코드이다.
- 파이썬에서는 값을 가져와서 그 값이 0이 아닌지 검사한 후 사용하는 패턴이 자주 발생
- count가 여러 번 쓰이는 경우를 해결하기 위해 많은...꼼수...가독성 최악..


왈러스 연산자를 이용해보면

In [2]:
# 이렇게 하면 깔끔하게 처리가 된 것 처럼 보인다.

if count := fresh_fruit.get('레몬', 0):
    make_lemonade(count)
else:
    out_of_stock()

사과의 경우 최소 네 개가 필요하다. 이럴 경우 ....

In [3]:
def make_cider(count):
    ...

if (count := fresh_fruit.get('사과', 0)) >= 4:
    make_lemonade(count)
else:
    out_of_stock()

위의 코드에 대하여
- if 문에서 대입 결과와 4를 비교하기 위해 대입식을 괄호로 둘러싸야 한다는 점이 중요
- 레모네이트 예제에서는 대입식이 다른 큰 식의 하위 식이 아니라, if 문의 조건에서 대입식 자체가 0이 아닌지 비교하는 데 쓰였으므로 괄호가 필요하지 않았다.
- 다른 식과 마찬가지로 가능하면 대입식 주변에 괄호를 쓰는 일을 피해야 한다.

조건에 따라 현재 위치를 둘러싸는 영역에 있는 변수에 값을 대입하고 그 변수를 바로 함수 호출에 사용하는 경우도 있다.
- 예를 들어 고객이 바나나 스무디를 주문했다고 하자.
- 스무디를 만들려면 바나나 슬라이스가 최소 두 개는 필요하고, 슬라이스가 부족하면 OutofBananas 예외를 발생시켜야 한다.

In [4]:
def slice_bananas(count):
    ...
    
class OutOfBananas(Exception):
    pass

def make_smoothies(count):
    ...
    
pieces = 0
count = fresh_fruit.get('바나나', 0)
if count >= 2:
    pieces = slice_bananas(count)

try:
    smoothies = make_smoothies(pieces)
except OutOfBananas:
    out_of_stock()

In [ ]:
# ver 2

count = fresh_fruit.get('바나나', 0)
if count >= 2:
    pieces = slice_bananas(count)
else:
    pieces = 0 # 좋은 방법은 아니다. 먼저 대입하는 방식을 선호한다.

try:
    smoothies = make_smoothies(pieces)
except OutOfBananas:
    out_of_stock()

In [ ]:
# 위의 코드를 한 줄로 줄인다면...

if (count := fresh_fruit.get('바나나', 0)) >= 2:
    pieces = slice_bananas(count)

여러 주스를 만든다고 한다고 할 때...

In [5]:
# 지저분한 코드

count = fresh_fruit.get('바나나', 0)

if count >= 2:
    pieces = slice_bananas(count)
    to_enjoy = make_smoothies(pieces)
else:
    count = fresh_fruit.get('사과', 0)
    if count >= 4:
        to_enjoy = make_cider(count)
    else:
        count = fresh_fruit.get('레몬', 0)
        if count:
            to_enjoy = make_lemonade(count)
        else:
            to_enjoy = "아무것도 없음"

In [6]:
# 이렇게 바꿀 수 있다.

if (count := fresh_fruit.get('바나나', 0)) >= 2:
    pieces = slice_bananas(count)
    to_enjoy = make_smoothies(pieces)
elif (count := fresh_fruit.get('사과', 0)) >= 4:
    pieces = slice_bananas(count)
elif (count := fresh_fruit.get('레몬', 0)):
    to_enjoy = make_smoothies(pieces)
else:
    to_enjoy = "아무것도 없음"

do/while 에 대한 접근

In [ ]:
# 코드가 반복적으로 호출 되고 있다. 

def pick_fruit():
    ...
    
def make_juice(fruit, count):
    ...
    
    
bottles = []
fresh_fruit = pick_fruit()

while fresh_fruit:
    for fruit, count in fresh_fruit.items():
        batch = make_juice(fruit, count)
        bottles.extend(batch)
    fresh_fruit = pick_fruit()

어떻게 해결해야 할까?
- 무한 루프 - 중간에서 끝내기 관용어를 사용하는 것

In [8]:
bottles = []

while True: # 무한 루프
    fresh_fruit = pick_fruit()
    if not fresh_fruit: # 중간에서 끝내기
        break
    
    for fruit, count in fresh_fruit.items():
        batch = make_juice(fruit, count)
        bottles.extend(batch)

왈러스 연산자를 사용하면 무한 루프 - 중간에서 끝내기 관용어의 필요성이 줄어든다.

In [ ]:
bottles = []

while fresh_fruit := pick_fruit():
    for fruit, count in fresh_fruit.items():
        batch = make_juice(fruit, count)
        bottles.extend(batch)

대입식을 사용해 중복을 줄일 수 있는 다른 상황도 많다.
- 대입식에서는 왈러스 연산자를 사용해 하나의 식 안에서 변수 이름에 값을 대입하면서 이 값을 평가할 수 있고, 중복을 줄일 수 있다.
- 대입식이 더 큰 식의 일부분으로 쓰일 때는 괄호로 둘러싸야 한다.
- 파이썬에서는 switch/case 문이나 do/while 루프를 쓸 수 없지만, 대입식을 사용하면 이런 기능을 더 깔끔하게 흉내 낼 수 있다.